### Building a Neural Network Keras Model for Network Intrusion Detection




In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier

2024-04-11 21:17:44.875676: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train = pd.read_csv('archive/Train_data.csv')
test = pd.read_csv('archive/Test_data.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25192 entries, 0 to 25191
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     25192 non-null  int64  
 1   protocol_type                25192 non-null  object 
 2   service                      25192 non-null  object 
 3   flag                         25192 non-null  object 
 4   src_bytes                    25192 non-null  int64  
 5   dst_bytes                    25192 non-null  int64  
 6   land                         25192 non-null  int64  
 7   wrong_fragment               25192 non-null  int64  
 8   urgent                       25192 non-null  int64  
 9   hot                          25192 non-null  int64  
 10  num_failed_logins            25192 non-null  int64  
 11  logged_in                    25192 non-null  int64  
 12  num_compromised              25192 non-null  int64  
 13  root_shell      

In [3]:
train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [4]:
def label_encoding(df):
    for col in df.columns:
        if df[col].dtype == 'object':
                label_encoder = LabelEncoder()
                df[col] = label_encoder.fit_transform(df[col])

label_encoding(train)
label_encoding(test)

train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,1,19,9,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,1
1,0,2,41,9,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,1
2,0,1,46,5,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,0
3,0,1,22,9,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,1
4,0,1,22,9,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1


In [5]:
train.drop(['num_outbound_cmds'], axis=1, inplace=True)
X = train.drop(['class'], axis=1)
y = train['class']

In [6]:
feature_list = ['src_bytes',
 'dst_bytes',
 'flag',
 'same_srv_rate',
 'diff_srv_rate',
 'dst_host_same_srv_rate',
 'dst_host_srv_count',
 'dst_host_srv_serror_rate',
 'logged_in',
 'count',
 'dst_host_same_src_port_rate',
 'dst_host_diff_srv_rate',
 'service',
 'protocol_type',
 'serror_rate']

X = X[feature_list]

X.head()

,src_bytes,dst_bytes,flag,same_srv_rate,diff_srv_rate,dst_host_same_srv_rate,dst_host_srv_count,dst_host_srv_serror_rate,logged_in,count,dst_host_same_src_port_rate,dst_host_diff_srv_rate,service,protocol_type,serror_rate
0,491,0,9,1.00,0.00,0.17,25,0.00,0,2,0.17,0.03,19,1,0.0
1,146,0,9,0.08,0.15,0.00,1,0.00,0,13,0.88,0.60,41,2,0.0
2,0,0,5,0.05,0.07,0.10,26,1.00,0,123,0.00,0.05,46,1,1.0
3,232,8153,9,1.00,0.00,1.00,255,0.01,1,5,0.03,0.00,22,1,0.2
4,199,420,9,1.00,0.00,1.00,255,0.00,1,30,0.00,0.00,22,1,0.0


In [7]:
X_train,X_test,Y_train,Y_test = train_test_split(X,y,train_size=0.70, random_state=42)
keras_model = Sequential()
keras_model.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu', input_dim = 15))
keras_model.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
keras_model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
keras_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
keras_model.fit(X_train, Y_train, batch_size = 10, epochs = 100)
train_predict = (keras_model.predict(X_train) > 0.5)
test_predict = (keras_model.predict(X_test) > 0.5)
model_predict = keras_model.predict(X_test)

Epoch 1/100


2024-04-11 21:17:49.095409: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1764/1764 [==============================] - 2s 874us/step - loss: 0.3058 - accuracy: 0.9182
Epoch 2/100
1764/1764 [==============================] - 2s 908us/step - loss: 0.1372 - accuracy: 0.9538
Epoch 3/100
1764/1764 [==============================] - 2s 854us/step - loss: 0.1665 - accuracy: 0.9601
Epoch 4/100
1764/1764 [==============================] - 2s 856us/step - loss: 0.1118 - accuracy: 0.9648
Epoch 5/100
1764/1764 [==============================] - 2s 854us/step - loss: 0.1038 - accuracy: 0.9647
Epoch 6/100
1764/1764 [==============================] - 2s 860us/step - loss: 0.1266 - accuracy: 0.9670
Epoch 7/100
1764/1764 [==============================] - 2s 858us/step - loss: 0.1004 - accuracy: 0.9669
Epoch 8/100
1764/1764 [==============================] - 2s 856us/step - loss: 0.0965 - accuracy: 0.9666
Epoch 9/100
1764/1764 [==============================] - 1s 849us/step - loss: 0.0946 - accuracy: 0.9687
Epoch 10/100
1764/1764 [==============================] - 2s 875us/

In [8]:
print ("Accuracy of the Keras model: ", metrics.accuracy_score(Y_test, test_predict))
print("Metrics of the classification report" "\n", metrics.classification_report(Y_test, test_predict)) 

Accuracy of the Keras model:  0.9793596189468113
Metrics of the classification report
               precision    recall  f1-score   support

           0       0.99      0.96      0.98      3516
           1       0.97      0.99      0.98      4042

    accuracy                           0.98      7558
   macro avg       0.98      0.98      0.98      7558
weighted avg       0.98      0.98      0.98      7558

